### 크롤링할 때(네이버만) 요약봇 버튼 누르고 요약내용 크롤링 해서 엑셀 'summary.xlsx'에 저장하기
### 엑셀파일에 textrank로 요약한 요약문을 저장하기
### 네이버 요약봇은 "3문장"으로 요약됨.

In [14]:
import urllib.request
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import kss

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os

class newsCrawler:
    def __init__(self):
        self.titleList=[]
        self.contentsList=[]
        self.imageList=[]
        self.dateList=[]
        
    # 네이버 뉴스홈
    def mainCrawl(self):    
        # 정치=100 경제=101 사회=102 생활/문화=103 세계=104 IT/과학=105
        # 실제 실행에서는 range(100, 106)으로 해야함.
        for category in range(101, 102):
            main_url = "https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1="+str(category)
            driver.get(main_url)
            
            # '헤드라인 더보기' 버튼이 있다면 누르기       
            self.showMore()
            driver.implicitly_wait(5)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')  
            
            # 헤드라인 가져오기
            self.subCrawl(soup, category)
        driver.quit()
    
    # 더보기버튼 클릭
    def showMore(self):
        try:
            while True:
                driver.find_element_by_xpath('//*[@id="main_content"]/div/div[2]/div[2]/div/a').click()
        except exceptions.ElementNotVisibleException as e:
            pass
        except Exception as e:
            pass
    
    # 헤드라인 뉴스 크롤링
    def subCrawl(self, soup, category):
        # 모든 헤드라인 뉴스 저장
        articles = soup.find_all('div', {'class': 'cluster_group _cluster_content'})
        sum_bots=[]
        sum1=[]
        sum2=[]
        sum3=[]
        
        for i in range(len(articles)):
            # 각 뉴스 본문에 있는 이미지와 이미지URL를 저장할 리스트
            images=[]
            imagesURL=[]
            
            # 본문 링크
            temp = articles[i].find_all('div', {'class': 'cluster_text'})[0]
            #print(temp.a.text)         
            conURL = temp.a.get('href')
            print(conURL)
            
            # 요약봇 3문장 크롤링
            driver.get(conURL)
            sum_bot=""
            
            try:
#                 while sum_bot=="":
#                     driver.find_element_by_xpath('//*[@id="main_content"]/div[1]/div[3]/div/div[3]/div[2]/div[1]/a').click()
#                     driver.implicitly_wait(30)
#                     sum_bot = driver.find_element_by_class_name("_contents_body").text
                
                driver.find_element_by_xpath('//*[@id="main_content"]/div[1]/div[3]/div/div[3]/div[2]/div[1]/a').click()
                element = WebDriverWait(driver, 100).until(
                    EC.text_to_be_present_in_element_value((By.CLASS_NAME, "_contents_body"))
                )
                sum_bot = element.text
                
                print(sum_bot)
                print(sum_bot.split('\n\n'))
                if len(sum_bot.split('\n\n'))<3:
                    pass
                else:
                    sum1.append(sum_bot.split('\n\n')[0])
                    sum2.append(sum_bot.split('\n\n')[1])
                    sum3.append(sum_bot.split('\n\n')[2])
                print(sum1)
            except exceptions.ElementNotVisibleException as e:
                pass
            except Exception as e:
                pass
          
            # 기사 제목, 내용 크롤링
            html2 = session.get(conURL,headers=headers).content
            soup2 = BeautifulSoup(html2, 'html.parser')
            
            summary = soup2.find('strong', {'class':'media_end_summary'})
            if summary==None:
                summary=""
            else:
                summary=summary.text
            content = soup2.find('div', id= "articleBodyContents").text.replace("\n"," ").replace(str(summary),"").replace("\t"," ").replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}"," ")
            title=soup2.find('h3',id="articleTitle").text
            
            #print(content)
            
        df2 = pd.DataFrame({'sum1':sum1, 'sum2':sum2, 'sum3':sum3})
        print(df)
        df2.to_excel('summary_bot.xlsx')
        
            
        
        
    
    def saveNewsToFile(self):
        f = open('result0319.txt', 'w', encoding='utf-8')
        for i in range(len(self.titleList)):
            f.write(self.titleList[i] + "\n")
            f.writelines(self.contentsList[i]+"\n")
            f.writelines(self.imageList[i])
            f.writelines("\n\n")
        f.close()
    
def cosine_sim(count):
    df =  pd.read_excel('news.xlsx')
    df = df.drop_duplicates(['title'], keep='first', ignore_index=True)
    # 모든 단어들의 빈도에 대하여 유사도를 계산하면 값이 너무 작게 나와서
    # max_features옵션으로 Tf-Idf의 크기를 줄인 다음 코사인 유사도를 계산함.
    tfidf = TfidfVectorizer(max_features=100)
    
    # max_feature는 tf-idf vector의 최대 feature를 설정해주는 파라미터입니다.
    # 머신러닝에서 feature란, 테이블의 컬럼에 해당하는 개념입니다. 또한 행렬의 열에 해당하는 것이기도 합니다.
    # TF-IDF 벡터는 단어사전의 인덱스만큼 feature를 부여받습니다.
    
    # doc: 기사 본문(문서)
    # tfidf_mat: 문서들을 벡터화한 
    doc = list(df['content'])
    tfidf_mat = tfidf.fit_transform(doc).toarray()
    #print(type(tfidf_mat))
    #print(tfidf_mat)
    #print(tfidf_mat.shape)
    
    # 소수점 4자리까지 반올림
    sim = np.round(cosine_similarity(tfidf_mat, tfidf_mat),4)
    #print(sim)
    #print(type(sim))
    #print(sim.shape)
    
    #print(sim)
    
    sim_scores = list(enumerate(sim[count]))
    #print(sim_scores)
    
    # 유사도가 높은 순서대로 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 상위 인덱스와 유사도 추출
    sim_scores = sim_scores[1:6]
    #print(sim_scores)
    
    # 원하는 기사와 유사한 기사 인덱스를 이용하여 제목 출력
    news_indices = [i[0] for i in sim_scores]
    result = df['title'].iloc[news_indices]
    print(result)
    print(tuple(result.index))
    
    df.set_index('date', inplace = True)
    df.to_excel('news.xlsx')
    
    return tuple(result.index)
    
        
        

session = requests.Session()

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}

#driver = webdriver.Chrome(r"C:\Users\LCM\Downloads\chromedriver_win32\chromedriver.exe")
driver = webdriver.Chrome(r"C:\Users\seenw\Downloads\chromedriver_win32\chromedriver.exe")

c=newsCrawler()
c.mainCrawl()
#cosine_sim(10)

https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=003&aid=0010474174
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=018&aid=0004917125
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=009&aid=0004786916
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=022&aid=0003576470
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=421&aid=0005322680


KeyboardInterrupt: 

In [ ]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

import kss


class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        # 불용어 불러오기
        self.stopwords = [line.rstrip('\n') for line in open('stopwords_korean3.txt', encoding = 'utf-8')]
    
    def text2sentences(self, text):
        #sentences = self.kkma.sentences(text)\
        sentences = kss.split_sentences(text)
        # 기사의 마지막 문장 1개 제외시킴
        sentences = sentences[0:len(sentences)-2]
        
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
                
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
                    if noun not in self.stopwords and len(noun) > 1]))
                
        return nouns


class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        # tfidf_mat의 크기는 문장수x단어수
#         print("tf-idf 행렬:")
#         print(tfidf_mat)
#         print(tfidf_mat.shape)
        
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
#         print("graph_sentence 행렬:")
#         print(self.graph_sentence)
#         print(self.graph_sentence.shape)
        # graph_sentence는 각 문장 벡터들간의 유사도 행렬이므로
        # 행렬의 크기는 문장수x문장수
        # 그래프로 그리면 노드가 문장수만큼 존재하고,
        # 이 그래프를 textrank 알고리즘의 입력으로 사용하여 각 문장의 점수를 구한다.
        # 점수가 높은 상위 n개를 선택하여 문서의 요약문으로 삼는다.
        
        
        #print(self.tfidf.get_feature_names())
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
    
    
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
                
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

    
    
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
            
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        #print(self.sent_graph)
        #print(self.sent_graph.shape)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
        #print(self.nouns)
        
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
    
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
            
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
            
        return keywords

# db와 연결 - db에 있는 기사 본문?요약?들을 tf-idf행렬로 벡터화 - 각 유사도 계산
# - 새로 크롤링한 기사들과 유사한 '최근' 기사 5개씩 db에 저장

# 아니면 db와 연결하지 말고 txt파일로 저장하는걸 추가로 구현한 다음에
# txt를 읽고 유사도를 계산하는게 더 나을수도

# 그럼 txt파일 or csv파일에 어떤 내용들을 저장할까
# 지금까지 크롤링한 기사본문/카테고리(100~105)/tf-idf행렬

# class Recommendation(object):
#     def __init__(self, text):
        
#     def cosine_sim(self):
        
#     def get_recommendation(self):
        